In [56]:
import pandas as pd
import numpy as np
import pvlib
from pvlib.location import Location
import sympy as sm
from scipy import integrate

def inner(x1: sm.Matrix,x2: sm.Matrix):
    '''
    Computes the inner product of two vectors of same length.
    '''

    return x1.dot(x2, conjugate_convention = 'right')

sm.MutableDenseMatrix.inner = inner
sm.ImmutableDenseMatrix.inner = inner

# Metoder

In [166]:
def solar_panel_projection(θ_s, ϕ_s, θ_p, ϕ_p):
    results = []
    for i in range(0,len(θ_s)):
        us = sm.Matrix([-np.sin(θ_s[i]) * np.cos(ϕ_s[i]),-np.sin(θ_s[i]) * np.sin(ϕ_s[i]), -np.cos(θ_s[i])])
        up = sm.Matrix([-1*np.sin(θ_p[i]) * np.cos(ϕ_p[i]), -np.sin(ϕ_p[i])*np.sin(θ_p[i]), -np.cos(θ_p[i])])
        #display(us, up)
        inprod = (inner(us,up))
        simpinprod = inprod.simplify()
        if(simpinprod < 0):
            results.append(0.0)
        else:
            results.append(simpinprod)
    return results

def fluxMinuteEntireDay(day, theta_p, phi_p):
    S0A=550
    L = 1.755
    B = 1.038

    zenithangs = np.deg2rad(np.array(solpos.loc[day].apparent_zenith))
    azimuthangs = np.deg2rad(np.array(solpos.loc[day].azimuth))
    invalidList = []
    for q in range(len(zenithangs)):
        if np.deg2rad(zenithangs[q])<0 or np.deg2rad(zenithangs[q])>(np.pi/2):
            invalidList.append(q)
    
    p_t = np.full(len(zenithangs),theta_p)
    p_p = np.full(len(zenithangs),phi_p)
 
    projList =  solar_panel_projection(zenithangs,azimuthangs,p_t,p_p)
    
    for index in invalidList:
        projList[index]=0
    for i in range(0,len(projList)):
        projList[i] = projList[i] * S0A

    
    return projList

def fluxToEffect(flux,maxFlux,maxEffect):

    effectPerFlux = maxEffect/maxFlux
    result  = []
    for i in range (len(flux)): 
        result.append(effectPerFlux*flux[i] if flux[i]<=maxFlux else maxEffect)
       # result.append(flux[i]/maxFlux*maxEffect if flux[i]<=maxFlux else maxEffect)
    return result

def KwtPerDay(integral):
    L=1.755
    B=1.038
    J=integral*L*B
    return(J/3600000)



# Sol Værdier setup

In [58]:
tidszone = "Europe/Copenhagen"
start_dato = "2024-04-01"
slut_dato = "2024-04-30"
delta_tid = "Min"  # "Min", "H",

valgt_dato = "2024-04-20"

# Definition of Location object. Coordinates and elevation of Amager, Copenhagen (Denmark)
site = Location(
    55.7861111111, 12.5230555556, tidszone, 10, "DTU (DK)"
)  # latitude, longitude, time_zone, altitude, name

# Definition of a time range of simulation
times = pd.date_range(
    start_dato + " 00:00:00", slut_dato + " 23:59:00", inclusive="left", freq=delta_tid, tz=tidszone
)
# Estimate Solar Position with the 'Location' object
solpos = site.get_solarposition(times)

# Beregninger

## Flux værdier for theta_s = 45, phi_p = 180

In [59]:
#zenithangs = np.array(solpos.loc[start_dato:slut_dato].zenith)
#azimuthangs = np.array(solpos.loc[start_dato:slut_dato].azimuth)
fluxting =np.array((fluxMinuteEntireDay(valgt_dato, 45,180)))
for i in range(0,len(fluxting)):
    print(fluxting[i])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [60]:
integral = integrate.simpson(fluxToEffect(fluxting,1000,365),dx=60)
#pr. m^2
display(integral)

5611692.89549471

## Fund af optimal vinkel d 20. april 2024, samt KwT og optimal vinkel

In [170]:
results = []
max = 0
angle = -1
for i in range(0,90):
        fluxting =np.array((fluxMinuteEntireDay(valgt_dato, i,180)))
        current = integrate.simpson(fluxToEffect(fluxting,1000,365),dx=60)
        current = KwtPerDay(current)
        results.append(current)
        if(current > max):
            max = current
            angle = i

print (max, "angle =",angle)

2.88270354439743 angle = 26
